In [ ]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
from datetime import datetime
from sqlalchemy import create_engine
import psycopg2
import time



def crawler_comex():
    link = 'https://www.gov.br/mdic/pt-br/assuntos/comercio-exterior/estatisticas/base-de-dados-bruta'
    try:
        resposta = rq.get(link)
        soup = BeautifulSoup(resposta.text, 'html.parser')

    except resposta.status_code != 200:  # type: ignore
        print("Erro!")
    soup = BeautifulSoup(resposta.text, 'html.parser')
    return soup
    

def extract_dim_table():
    soup = crawler_comex()
    links = soup.find_all('table',class_ = 'plain')[6]('a')
    link_dim = [x.get('href') for x in links if x.get_text() == 'NCM']
    return link_dim[0]

def extract_fato_table():
    soup = crawler_comex()
    #Exportaçao
    export_links = soup.find_all('table',class_ = 'plain')[0]('a',class_='external-link')

    #Importaçãos
    import_links = soup.find_all('table',class_ = 'plain')[1]('a',class_='external-link')

    # Obter o ano atual
    ano_atual = datetime.now().year


    link_base_de_dados_export = [x.get('href') for x in export_links if x.get_text() == str(ano_atual)]
    link_base_de_dados_import = [x.get('href') for x in import_links if x.get_text() == str(ano_atual)]
    return link_base_de_dados_export[0],link_base_de_dados_import[0]

def create_stage():
    dim_folder_path_stage = r'C:\Users\GabrielLeal\OneDrive - Grupo Portfolio\Área de Trabalho\DataOps\landing\dimensao'
    fat_folder_path_stage = r'C:\Users\GabrielLeal\OneDrive - Grupo Portfolio\Área de Trabalho\DataOps\landing\fato'

    link_export,link_import = extract_fato_table()
    link_dim =  extract_dim_table()


    fat_export = pd.read_csv(link_export,sep =';',encoding='ISO-8859-1')
    fat_import = pd.read_csv(link_import,sep =';',encoding='ISO-8859-1')
    dim_comex = pd.read_csv(link_dim, sep=';',encoding='ISO-8859-1')

    dim_comex.to_excel(dim_folder_path_stage+'/dim_landing.xlsx',index=False)
    print("Dimensão Comex adicionada na landing zone")

    fat_export.to_excel(fat_folder_path_stage+'/fat_export_landing.xlsx',index=False)
    print("Fato Comex exportação adicionada na landing zone")
    
    fat_import.to_excel(fat_folder_path_stage+'/fat_import_landing.xlsx',index=False)
    print("Fato Comex importação adicionada landing zone")


    



if __name__ == "__main__":
    create_stage()
    print('Stage de dados criada!')


Dimensão Comex adicionada na Stage
Fato Comex exportação adicionada na Stage
Fato Comex importação adicionada na Stage
Stage de dados criada!
